# Bla

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from scipy.optimize import minimize 
from datetime import timedelta
import datetime as dt
from IPython.display import Image
import requests

In [32]:
filename = 'data/brazil_covid19_06_11.csv'
df_ = pd.read_csv(filename)

In [33]:
def delta(df_conf):

    list_ = []
    list_.append(0)

    for j in range(len(df_conf) - 1):

        list_.append(df_conf[j+1] - df_conf[j])

    return list_

alpha = 30.
alpha2 = 7.

In [34]:
states_ = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RS', 'RO',  'SC', 'SP', 'SE']

states_2 = ['Blumenau', 'Sao_Carlos', 'Sao_Paulo']

states_3 = ['MS', 'RR', 'TO', 'RN']

#states_4 = ['RJ']

In [37]:
state_or_city = 'city'
fit_until = '06-10'

for j in states_2:
    name = str(j)
    print(name)

    path = 'results/dfs/'
    path_ = 'results/dfs/csv/'

    file_ = 'df_ICU_' + state_or_city + '_' + name + '_fit_until_' + fit_until


    #############################################
    df2 = pd.read_pickle(path + file_ + '.pkl')
    df2 = df2.drop(columns=['n_mean', 'n_std', 'n_mean_ICU_PRIVATE', 'n_std_ICU_PRIVATE'])
    #############################################

    #state_or_city = 'city'
    df = df_[ df_['place_type'] == state_or_city]
    df = df[ df[state_or_city] == name ]
    df_I = df.groupby('date')[['confirmed', 'confirmed_per_100k_inhabitants']].sum()
    df_I.index = pd.to_datetime(df_I.index)

    df_I['growth_rate'] = delta(df_I['confirmed'])
    # Exponential weight
    df_I['growth_rate_EWM'] = df_I['growth_rate'].ewm(com= alpha).mean()
    # Rate
    df_I['growth_accel'] = delta(df_I['growth_rate_EWM'])
    # Exponential weight
    df_I['growth_accel_EWM'] = df_I['growth_accel'].ewm(com= alpha2).mean()

    df_I['growth_rate_EWM_NORM_(%)'] = 100*df_I['growth_rate_EWM'] / df_I['confirmed']
    df_I['growth_accel_EWM_NORM_(%)'] = 100*df_I['growth_accel_EWM'] / df_I['confirmed']

    df_I['growth_rate_NORM_(%)'] = 100*df_I['growth_rate'] / df_I['confirmed']
    df_I['growth_accel_NORM_(%)'] = 100*df_I['growth_accel'] / df_I['confirmed']

    #############################################

    df_END = df2.join(df_I)


    df_END.to_csv(path_ + file_ + '_.csv', index= True, sep= ',')

Blumenau
Sao_Carlos
Sao_Paulo
